In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import graphviz as gv
import json
from Bio import SeqIO

In [2]:
import subprocess

In [ ]:
smash = pd.read_csv("antismash_db_results.csv", sep="\t")
smash[smash["NCBI accession"] == "NC_010612.1"] 
counts = smash["NCBI accession"].value_counts()
above_10_index = counts[counts > 10].index
smash = smash[smash["NCBI accession"].isin(above_10_index)]
smash["NCBI accession"] = smash["NCBI accession"].apply(lambda row: row.split(".")[0])
smash_dict = dict()
for accession, df in smash.groupby("NCBI accession"):
    smash_dict[accession] = df[["From", "To"]].to_numpy()

In [146]:
x = pd.read_csv("top_3.features_with_pafm.tsv", sep="\t") #.sort_values(["genome_id", "sequence_id", "start"])
# x.dropna(subset=["pgfam_id"]).to_csv("top_3.features_with_pafm.tsv", sep="\t")
ind = x.index
ind_sort = x.sort_values(["genome_id", "accession", "start"]).index
x[(ind != ind_sort)]["genome_id accession annotation feature_type start end pgfam_id".split()]
# x.dropna(subset=["pgfam_id"]).to_csv("top_3.features_with_pafm.tsv", sep="\t", index=False)
# x

,genome_id,accession,annotation,feature_type,start,end,pgfam_id


In [143]:
records = [r for r in SeqIO.parse("/sfs/lustre/bahamut/scratch/jho5ze/bionets/BGCs/genera/Mycobacterium/jsonhfasta/first_pass.jsonhfasta", "fasta")]
seq_dict = [json.loads(r.description.split(None, 1)[-1]) for r in records]
seq_dict = {x["md5"]:sorted([i for i in x.setdefault("sequence_domain", [{"source":"none", "start":"0"}]) if i["source"] == "pfam"], key = lambda x: int(x["start"])) for x in seq_dict}

def gen_pfam_graph_input(source="top_3.features.tsv", destination = "top_3.features_with_pafm.tsv", )
    with open(source) as src:
        with open(destination, "w") as dest:
            for ix, line in enumerate(src.readlines()):
                vals = line.strip().split("\t")
                if ix == 0:
                    names = vals
                    name_ids = {name:ix for ix, name in enumerate(names)}
                vals = {name:vals[ix] for ix, name in enumerate(names)}
                md5 = vals["aa_sequence_md5"]
                pgfam = vals["pgfam_id"]

                #This means that the pgfam is NAN and should be skipped
                if len(pgfam) < 2:
                    continue

                dest.write(line)

                try:
                    pfams = seq_dict[md5]
                except:
                    continue
                if len(pfams) > 0:
                    for pfam in pfams:
                        pfam_line = line.split("\t")
                        pfam_line[name_ids["start"]] = str(int(vals['start']) + int(pfam['start']))
                        pfam_line[name_ids["end"]] = str(int(vals['start']) + int(pfam['end']))
                        pfam_line[name_ids["na_length"]] = str(int(pfam['end']) - int(pfam['start']))
                        pfam_line[name_ids["pgfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["figfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["plfam_id"]] = pfam["sequence_domain"]
                        pfam_line[name_ids["annotation"]] = "CUSTOM"
                        dest.write(("\t".join(pfam_line)))



17234


In [14]:
x = json.loads(records[0].description.split(None, 1)[-1])

In [22]:
[i for i in x["sequence_domain"] if i["source"] == "pfam"]

[{'end': 471,
  'name': 'Peptidase M28',
  'sequence_domain': 'PF04389',
  'start': 252,
  'ipr_id': 'IPR007484',
  'source': 'pfam'},
 {'end': 229,
  'name': 'PA domain',
  'sequence_domain': 'PF02225',
  'start': 140,
  'ipr_id': 'IPR003137',
  'source': 'pfam'}]

In [11]:
# !module load muscle

In [ ]:
def generic_msa(strings):
    """
    Implement from here: http://readiab.org/book/0.1.2/2/3#1.2
    """
    pass

In [2]:
def reannotate_graph(graph, feature_map):
    """
    Feature map should translate the md5 hash to new features
    and should include start and stop per feature, or ordered 
    features to generate a new node resolution
    """
    pass

In [135]:
# g = "../genera/Acaryochloris/graphs/Acaryochloris_gids.gexf"
# g = "/sfs/lustre/bahamut/scratch/jho5ze/bionets/patric_genera/fork/pangenome_graphs/Mycobacterium_all_md5s_and_coords.gexf"
g = "/sfs/lustre/bahamut/scratch/jho5ze/bionets/BGCs/genera/Mycobacterium/graphs/top_3.features_with_pafm.gexf"
g = nx.readwrite.read_gexf(g)

In [137]:
json.loads(g.nodes['1000']["features"].replace('""', '"'))

{'info': {'1078013.3': {'AGAQ01000593': [{'start': 85,
     'end': 849,
     'md5': None}]},
  '1041522.3': {'NZ_AFVW01000003': [{'start': 674004,
     'end': 674759,
     'md5': None}]}},
 '1078013.3': {'AGAQ01000593': ['fig|1078013.3.peg.4027']},
 '1041522.3': {'NZ_AFVW01000003': ['fig|1041522.3.peg.2271']}}

In [149]:
# json.loads(g.nodes['6532']["features"].replace('""', '"'))
g.nodes['1000']["family"]

'PGF_00930759'

In [8]:
#g.nodes['4257']
feature_dict = [(i, json.loads(g.nodes[i]["features"].replace('""', '"'))) for i in g.nodes]
feature_dict.sort(key=lambda tup: int(tup[0]))

In [9]:
feature_dict[0]

('0',
 {'146020.3': {'BCSX01000015': ['fig|146020.3.peg.1203']},
  '244292.6': {'LDPP01000030': ['fig|244292.6.peg.3180']},
  '1545691.3': {'JRMK01000078': ['fig|1545691.3.peg.2687']},
  '1299333.3': {'JAOC01000002': ['fig|1299333.3.peg.101']},
  '1209984.3': {'CCBF010000001': ['fig|1209984.3.peg.943']},
  '216594.6': {'NC_010612': ['fig|216594.6.peg.1106']},
  '1214102.3': {'ALQB01000212': ['fig|1214102.3.peg.5706']},
  '1078020.3': {'AGVE01000046': ['fig|1078020.3.peg.2325']},
  '233413.5': {'NC_002945': ['fig|233413.5.peg.792']},
  '1157943.3': {'AREU01000003': ['fig|1157943.3.peg.2908']},
  '1078013.3': {'AGAQ01000331': ['fig|1078013.3.peg.2529']},
  '710421.3': {'CP003053': ['fig|710421.3.peg.957']},
  '512402.8': {'LZLB01000061': ['fig|512402.8.peg.2847']},
  '710685.3': {'CP003169': ['fig|710685.3.peg.699']},
  '487514.18': {'LLXQ01000012': ['fig|487514.18.peg.623']},
  '487514.3': {'BBKA01000003': ['fig|487514.3.peg.180']},
  '1771.11': {'CP014475': ['fig|1771.11.peg.1293']},
 

In [22]:
feature_dict[0]

('0',
 {'start': 1991379,
  '329726.14': {'NC_009925': ['fig|329726.14.peg.1809']},
  'end': 1991720,
  '310037.4': {'AFEJ01000001': ['fig|310037.4.peg.4']},
  'md5': '358b5099bb71279401aa37fd236ac37e'})

In [5]:
g = "test.gexf"
g = nx.readwrite.read_gexf(g)
g.nodes['6532']

{'diversity': 1.0,
 'features': '{"start": 12622, "end": 14079, ""310037.4"": {""AFEJ01000362"": [""fig|310037.4.peg.6261""]}, "md5": ""29fcfc37edf97bb915a237d227cfc1ab""}',
 'family': '"PGF_08225224"',
 'alternate': 0,
 'conflict': 0,
 'viz': {'color': {'r': 153, 'g': 153, 'b': 153, 'a': 1.0},
  'size': 10.0,
  'position': {'x': -2981.6096, 'y': -4754.027, 'z': 0.0}},
 'label': '"hypothetical protein"'}